<a href="https://colab.research.google.com/github/metaphorz/v-diffusion-pytorch/blob/master/MP_v_diffusion_images512x512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Parameter v-diffusion-pytorch
### This is Katherine Crowson's v-diffusion-pytorch repository with some additions
### P. Fishwick, 01/17/22
1. Center installation in Google Drive by mounting it.
2. First run: the installation - store in Drive
3. First run: the pretrained model - store in Drive
4. A loop over a range of skip_timestep (st) and seed values (bottom of notebook) -- this allows you to "batch" run the notebook to create multiple images. There is a --batch_size option in cfg_sample.py and this is NOT used.
5. A collage option -- makes rectangular collages e.g. 3x3, 6x6, 4x7, 2x4
6. Storing resulting images in a zip file and then downloading

This notebook creates a "multiparam-v-diffusion-pytorch" directory within Google Drive.

In [ ]:
# Load Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#
# discover the assigned GPU
#
!nvidia-smi -L

In [ ]:
# install requirements
!pip install ftfy regex requests tqdm

In [ ]:
 # obtain v-diffusion-pytorch
# run this one time
root = '/content/drive/MyDrive/multiparam-v-diffusion-pytorch'
import os
if (not os.path.isdir(root)):
  !git clone --recursive https://github.com/metaphorz/v-diffusion-pytorch {root}
else:
  print(root+' exists')

In [ ]:
#
# Download the diffusion model
# The model list (checkpoints) can be found https://github.com/crowsonkb/v-diffusion-pytorch#model-checkpoints
# SHA-256: 4fc95ee1b3205a3f7422a07746383776e1dbc367eaf06a5b658ad351e77b7bda
#
# Save pretrained model in Google Drive for first run (v-diffusion-pytorch/ should not pre-exist!)
import os
if (not os.path.isdir(root+'/checkpoints')):
  !mkdir {root}/results
  !mkdir {root}/checkpoints
  !curl -L https://v-diffusion.s3.us-west-2.amazonaws.com/cc12m_1_cfg.pth > {root}/checkpoints/cc12m_1_cfg.pth
else:
  print('pretrained model already downloaded')
# From original 
# !curl -L https://v-diffusion.s3.us-west-2.amazonaws.com/cc12m_1_cfg.pth > {root}/checkpoints/cc12m_1_cfg.pth

In [ ]:
%cd {root}

In [ ]:
# Run inference loop given the init image for each st and seed value
# Edit the st and range values
import numpy as np
#
# Specify number of skip_timesteps
# in between st_start and st_stop
#
st_num = 1
st_start = 0.85
st_stop = 0.90
st_step = (st_stop - st_start)/st_num
st_list = np.arange(st_start, st_stop, st_step)
st_values = list(st_list)
print('# of st_values ', len(st_values))
#
# Specify number of seeds
# starting at seed_start
#
seed_num = 9
seed_start = 0
seed_stop = seed_start + seed_num
seed_step = 1
seed_list = np.arange(seed_start, seed_stop, seed_step)
seed_values = list(seed_list)
print('# of seeds ', seed_num)
print('# of generated images ', st_num*seed_num)
print('Image Generation ...')
for st_num in st_values:
  for seed_num2 in seed_values:
    # enter your own init image
    !python cfg_sample.py --init /content/drive/MyDrive/images/abandoned2.jpg -st $st_num --size 512 512 --seed $seed_num2

In [ ]:
# save all images in a zip
!zip {root}/images.zip {root}/results/*.png

In [ ]:
# download images.zip
# sometimes this gets stuck, so it is commented out
# instead, click on the images.zip file in Drive under multiparam-v-diffusion-pytorch
# and download the zip file
#from google.colab import files
#files.download(root+'/images.zip')

In [ ]:
# Optional: Google Drive results/ cleaning
# clean your results folder between notebook runs
# once images.zip has downloaded
#!rm {root}/images.zip
#!rm {root}/results/*.png

In [ ]:
%cd {root}/results

In [ ]:
# Generate a collage of all images
# and store in multiparam-v-diffusion-pytorch/
# Code adapted from: https://holypython.com/python-pil-tutorial/creating-photo-collages/

from PIL import Image, ImageDraw
import glob
lst = sorted(glob.glob("*.png"))
print('check list length ',len(lst))
# to make the collage, you need num_x * num_y .png files
# need not be square
# 9 images to make a 3x3 collage
# 36 images to make a 6x6 collage
# 500 pixels for each side of square component images
num_x = 3
num_y = 3
print('generating a ',num_x,' * ',num_y,' collage')
# choose this to modify the size of each square in the collage
# side_square = 200 # for smaller collage image
side_square = 512 # for the largest collage image


collage = Image.new("RGBA", (side_square*num_x,side_square*num_y), color=(255,255,255,255))

file_num = 0
for j in range(0,num_y*side_square,side_square):
    for i in range(0,num_x*side_square,side_square):
        image_file = str(lst[file_num])
        photo = Image.open(image_file).convert("RGBA")
        photo = photo.resize((side_square,side_square))          
        collage.paste(photo, (i,j))
        file_num += 1

display(collage)
collage.save(root+"/collage.png")
